In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [8]:
df = pd.read_csv('./datasets/wdbc.csv', header=None)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [9]:
X = df.iloc[:, 2:].values
y = df.iloc[:, 1]

le = LabelEncoder()
y = le.fit_transform(y)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y, random_state=1)

In [14]:
pipe_lr = make_pipeline (
    StandardScaler(),
    PCA(n_components=3),
    LogisticRegression()
)

In [27]:
# K-fold cross-validation implementation:

import numpy as np
from sklearn.model_selection import StratifiedKFold


kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
scores = []

for iteration, (train, val) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[val], y_train[val])
    scores.append(score)
    print('Fold: %2d, Class dist: %s, Validation Acc: %.2f' % (iteration+1, np.bincount(y_train[train]), score))

Fold:  1, Class dist: [256 153], Validation Acc: 0.89
Fold:  2, Class dist: [256 153], Validation Acc: 0.96
Fold:  3, Class dist: [256 153], Validation Acc: 0.98
Fold:  4, Class dist: [256 153], Validation Acc: 0.93
Fold:  5, Class dist: [256 153], Validation Acc: 0.87
Fold:  6, Class dist: [257 153], Validation Acc: 0.98
Fold:  7, Class dist: [257 153], Validation Acc: 0.93
Fold:  8, Class dist: [257 153], Validation Acc: 0.93
Fold:  9, Class dist: [257 153], Validation Acc: 0.98
Fold: 10, Class dist: [257 153], Validation Acc: 0.96


In [29]:
print('Average Accuracy: %.3f +/- %.4f' % (np.mean(scores), np.std(scores)))

Average Accuracy: 0.941 +/- 0.0350


In [30]:
# K-fold cross validation scorer:
from sklearn.model_selection import cross_val_score

scores = cross_val_score (
    estimator=pipe_lr,
    X=X_train,
    y=y_train,
    cv=10,
    n_jobs=2
)

print('CV Scores: %s' % (scores))

CV Scores: [0.89130435 0.95652174 0.97826087 0.93478261 0.86956522 0.97777778
 0.93333333 0.93333333 0.97777778 0.95555556]


In [31]:
print('Average Accuracy: %.3f +/- %.4f' % (np.mean(scores), np.std(scores)))

# An extremely useful feature of Kfold cross validation scorer is that we can evaluate the performance
# of different folds accross multiple CPU's on our machine by specify the number of cores as an argument
# to the parameter n_jobs

Average Accuracy: 0.941 +/- 0.0350
